In [19]:
%config InlineBackend.figure_formats = ['svg']
import sys
sys.path.insert(1, '../')

import matplotlib.pyplot as plt
from os import path

from deployment_instance.Result import ExperimentResult
from src.ingest_data import ingest_experiment_results
from src.query_data import get_exfiltration_time_df, get_data_exfiltration_cdf, total_control_host_capture_times
from statistics import mean
import seaborn as sns
# set seaborn font size
sns.set_theme(font_scale=1.5)
sns.set_style("whitegrid", {'axes.grid' : False})

import pandas as pd

In [21]:
# Equifax d10_h0
equifax_standalone_d10_h0 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/equifax/equifax_standalone_d10_h0'))
dfs_standalone_d10_h0 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/dfs/dfs_standalone_d10_h0'))
targeted_standalone_d10_h0 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/targeted/targeted_standalone_d10_h0'))
persistentdfs_standalone_d10_h0 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/persistent_dfs/persistentdfs_standalone_d10_h0'))
darkside_standalone_d10_h0 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/darkside/darkside_standalone_d10_h0'))

equifax_standalone_d10_data = {}
equifax_standalone_d10_data.update(equifax_standalone_d10_h0)
equifax_standalone_d10_data.update(dfs_standalone_d10_h0)
equifax_standalone_d10_data.update(targeted_standalone_d10_h0)
equifax_standalone_d10_data.update(persistentdfs_standalone_d10_h0) 
equifax_standalone_d10_data.update(darkside_standalone_d10_h0)

# Equifax d10_h50
equifax_standalone_d10_h50 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/equifax/equifax_standalone_d10_h50'))
dfs_standalone_d10_h50 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/dfs/dfs_standalone_d10_h50'))
# targeted_standalone_d10_h50 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/targeted/targeted_standalone_d10_h50'))
persistentdfs_standalone_d10_h50 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/persistent_dfs/persistentdfs_standalone_d10_h50'))
# darkside_standalone_d10_h50 = ingest_experiment_results(path.join('data/EquifaxLarge/standalone/darkside/darkside_standalone_d10_h50'))

equifax_standalone_d10_h50_data = {}
equifax_standalone_d10_h50_data.update(equifax_standalone_d10_h50)
equifax_standalone_d10_h50_data.update(dfs_standalone_d10_h50)
# equifax_standalone_d10_h50_data.update(targeted_standalone_d10_h50)
equifax_standalone_d10_h50_data.update(persistentdfs_standalone_d10_h50)
# equifax_standalone_d10_h50_data.update(darkside_standalone_d10_h50)

# Equifax reactive simple
# equifax_reactive_simple = ingest_experiment_results(path.join('data/EquifaxLarge/reactive/equifax/equifax_reactive_simple'))


In [22]:
equifax_standalone_d10_data_times = get_exfiltration_time_df(equifax_standalone_d10_data, 48)
equifax_standalone_d10_data_times.groupby('experiment').agg({'time_per_file': ['mean', 'std'], 'percent_files_exfiltrated': ['mean', 'std']})

time_per_file            \
                                         mean       std   
experiment                                                
baseline_standalone_d10_h0           0.883508  0.041200   
darkside_standalone_d10_h0           0.884415  0.011239   
dfs_standalone_d10_h0                4.802488  0.935535   
persistentdfs_standalone_d10_h0      7.738779  2.484619   
targeted_standalone_d10_h0           0.000000  0.000000   

                                percent_files_exfiltrated            
                                                     mean       std  
experiment                                                           
baseline_standalone_d10_h0                     100.000000  0.000000  
darkside_standalone_d10_h0                     100.000000  0.000000  
dfs_standalone_d10_h0                           31.250000  4.885850  
persistentdfs_standalone_d10_h0                 20.833333  8.715209  
targeted_standalone_d10_h0                       0.000000  0.000000

In [23]:
equifax_standalone_d10_h50_data_times = get_exfiltration_time_df(equifax_standalone_d10_h50_data, 48)
equifax_standalone_d10_h50_data_times.groupby('experiment').agg({'time_per_file': ['mean', 'std'], 'percent_files_exfiltrated': ['mean', 'std']})

time_per_file            \
                                          mean       std   
experiment                                                 
baseline_standalone_d10_h50           0.918099  0.059972   
dfs_standalone_d10_h50                0.000000  0.000000   
persistentdfs_standalone_d10_h50      0.000000  0.000000   

                                 percent_files_exfiltrated       
                                                      mean  std  
experiment                                                       
baseline_standalone_d10_h50                          100.0  0.0  
dfs_standalone_d10_h50                                 0.0  0.0  
persistentdfs_standalone_d10_h50                       0.0  0.0

In [24]:
# ICS d10_h0
equifax_standalone_d10_h0 = ingest_experiment_results(path.join('data/ICS/standalone/equifax/equifax_standalone_d10_h0'))
dfs_standalone_d10_h0 = ingest_experiment_results(path.join('data/ICS/standalone/dfs/dfs_standalone_d10_h0'))
targeted_standalone_d10_h0 = ingest_experiment_results(path.join('data/ICS/standalone/targeted/targeted_standalone_d10_h0'))
persistentdfs_standalone_d10_h0 = ingest_experiment_results(path.join('data/ICS/standalone/persistent_dfs/persistentdfs_standalone_d10_h0'))
darkside_standalone_d10_h0 = ingest_experiment_results(path.join('data/ICS/standalone/darkside/darkside_standalone_d10_h0'))

ics_standalone_d10_h0_data = {}
ics_standalone_d10_h0_data.update(equifax_standalone_d10_h0)
ics_standalone_d10_h0_data.update(dfs_standalone_d10_h0)
ics_standalone_d10_h0_data.update(targeted_standalone_d10_h0)
ics_standalone_d10_h0_data.update(persistentdfs_standalone_d10_h0)
ics_standalone_d10_h0_data.update(darkside_standalone_d10_h0)

In [25]:
ics_standalone_d10_data_times = total_control_host_capture_times(ics_standalone_d10_h0_data, 'ics')
ics_standalone_d10_data_times.groupby('experiment').agg({
    'percent_hosts_infected': ['mean', 'std'],
    'time_per_host': ['mean', 'std']
    })

/home/brian/Perry/data_analysis/src/query_data.py:242: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[df.shape[0]] = [


percent_hosts_infected             \
                                                  mean        std   
experiment                                                          
darkside_standalone_d10_h0                        84.0  35.777088   
dfs_standalone_d10_h0                            100.0   0.000000   
equifax_standalone_d10_h0                          0.0   0.000000   
persistentdfs_standalone_d10_h0                  100.0   0.000000   
targeted_standalone_d10_h0                       100.0   0.000000   

                                time_per_host            
                                         mean       std  
experiment                                               
darkside_standalone_d10_h0          10.717161  2.431498  
dfs_standalone_d10_h0                8.421197  1.237347  
equifax_standalone_d10_h0                 NaN       NaN  
persistentdfs_standalone_d10_h0      9.018739  0.646385  
targeted_standalone_d10_h0          12.312090  2.657219